In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directoryA
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/phones-data/cleaned_parallel_phones_val_phones.txt
/kaggle/input/phones-data/cleaned_parallel_phones_test_phones.txt
/kaggle/input/phones-data/cleaned_parallel_phones_test_hindi.txt
/kaggle/input/phones-data/cleaned_parallel_phones_train_hindi.txt
/kaggle/input/phones-data/cleaned_parallel_phones_val_hindi.txt
/kaggle/input/phones-data/cleaned_parallel_phones_train_phones.txt


In [2]:
import os
from collections import Counter

def get_unique_source_tokens(filepaths):
    """Reads source files and returns unique whitespace-separated tokens."""
    all_tokens = Counter()
    special_tokens = {"<WB>"} # Treat <WB> separately if desired
    for filepath in filepaths:
        if not os.path.exists(filepath):
            print(f"Warning: File not found - {filepath}")
            continue
        with open(filepath, "r", encoding="utf-8") as f:
            for line in f:
                # Split by whitespace, add individual characters/symbols if needed
                tokens_in_line = line.strip().split()
                all_tokens.update(tokens_in_line)

    # Exclude special tokens if handled separately later,
    # or keep them if you want to ensure they are added
    unique_tokens = list(all_tokens.keys())
    # Optionally filter out standard alphanumeric if not needed
    # unique_tokens = [t for t in unique_tokens if not t.isalnum() and t not in special_tokens]
    return unique_tokens

# List your source data files
source_files = ["/kaggle/input/phones-data/cleaned_parallel_phones_train_phones.txt","/kaggle/input/phones-data/cleaned_parallel_phones_val_phones.txt"] # Add test.src if you have one
unique_phones = get_unique_source_tokens(source_files)

# Optional: Print to review - you might want to refine this list
print(f"Found {len(unique_phones)} unique tokens in source files.")
print(unique_phones) # Uncomment to see the list

Found 62 unique tokens in source files.
['ʋ', 'ɛ', 'd̪', 'ə', '<WB>', 'k', 'm', 'p', 'ɲ', 'i', 'z', 'f', 'n', 'dʒ', 'ɪ', 'ʃ', 'oː', 'eː', 'ɒː', 'ɒ', 'ʈ', 'l', 's', 'ɹ', 'c', 'bʲ', 'ʈʲ', 'tʃ', 'a', 'mʲ', 'ŋ', 'ʎ', 'ɖ', 'ɜː', 'fʲ', 'ɑː', 'b', 'iː', 'ɡ', 'j', 'ʉː', 'pʲ', 'aː', 'aj', 'ɛː', 'ɾ', 'ɑ', 'ɟʷ', 'aw', 'h', 'ʊ', 'ʉ', 'cʷ', 'ɟ', 'kʷ', 't̪', 'ʈʷ', 'ç', 'ʒ', 'ɔj', 'ɜ', 'w']


In [3]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Use the multilingual model checkpoint
model_checkpoint = "facebook/mbart-large-50-many-to-many-mmt"
target_lang_code = "hi_IN" # Hindi
source_lang_code = "en_XX" # Pragmatic choice for MBart

print(f"Loading tokenizer: {model_checkpoint}")
tokenizer = MBart50TokenizerFast.from_pretrained(
    model_checkpoint,
    src_lang=source_lang_code,
    tgt_lang=target_lang_code
)

print(f"Loading model: {model_checkpoint}")
model = MBartForConditionalGeneration.from_pretrained(model_checkpoint)

print(f"Original tokenizer size: {len(tokenizer)}")
print(f"Original model embeddings size: {model.config.vocab_size}")

2025-05-05 16:17:45.991175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746461866.180889      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746461866.231945      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading tokenizer: facebook/mbart-large-50-many-to-many-mmt


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

Loading model: facebook/mbart-large-50-many-to-many-mmt


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

Original tokenizer size: 250054
Original model embeddings size: 250054


In [4]:
import os
from collections import Counter

def get_unique_source_tokens(filepaths):
    """Reads source files and returns unique whitespace-separated tokens."""
    all_tokens = Counter()
    for filepath in filepaths:
        if not os.path.exists(filepath):
            print(f"Warning: File not found - {filepath}")
            continue
        with open(filepath, "r", encoding="utf-8") as f:
            for line in f:
                tokens_in_line = line.strip().split()
                all_tokens.update(tokens_in_line)
    unique_tokens = list(all_tokens.keys())
    return unique_tokens

# List your source data files
source_files = ["/kaggle/input/phones-data/cleaned_parallel_phones_train_phones.txt","/kaggle/input/phones-data/cleaned_parallel_phones_val_phones.txt"] # Add test.src if you have one
unique_phones_to_add = get_unique_source_tokens(source_files)

print(f"Found {len(unique_phones_to_add)} unique tokens in source files to potentially add.")

# Add ALL unique tokens found.
# `add_tokens` will ignore any tokens already present in the vocabulary.
# Setting special_tokens=False means they are treated as regular vocabulary items,
# which is usually correct for phonetic symbols unless one needs very specific handling.
# '<WB>' will be added as a regular token if it doesn't exist.
if unique_phones_to_add:
    print(f"Attempting to add {len(unique_phones_to_add)} unique source tokens to the tokenizer...")
    num_added = tokenizer.add_tokens(unique_phones_to_add, special_tokens=False)
    print(f"Successfully added {num_added} new tokens (tokens already present are not re-added).")
else:
    print("No unique tokens found in source files to add.")

print(f"New tokenizer size: {len(tokenizer)}")


# --- CRITICAL: Resize Model Embeddings AFTER adding tokens ---
print("Resizing model embeddings...")
model.resize_token_embeddings(len(tokenizer))

# Verify the model's config reflects the new size
print(f"New model embedding matrix size: {model.get_input_embeddings().weight.shape[0]}")
# It's good practice to assert this worked
assert model.get_input_embeddings().weight.shape[0] == len(tokenizer), "Model embedding size doesn't match new tokenizer size!"

# --- Rest of your script continues below (Data Loading, Preprocessing, Training Setup...) ---

Found 62 unique tokens in source files to potentially add.
Attempting to add 62 unique source tokens to the tokenizer...
Successfully added 62 new tokens (tokens already present are not re-added).
New tokenizer size: 250079
Resizing model embeddings...


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


New model embedding matrix size: 250079


In [5]:

!pip install datasets
from datasets import load_dataset, DatasetDict, Dataset

# --- Load Raw Text Data --- (Assuming files exist)
print("Loading raw text data...")
train_src_lines = [line.strip() for line in open("/kaggle/input/phones-data/cleaned_parallel_phones_train_phones.txt", "r", encoding="utf-8")]
train_tgt_lines = [line.strip() for line in open("/kaggle/input/phones-data/cleaned_parallel_phones_train_hindi.txt", "r", encoding="utf-8")]
val_src_lines = [line.strip() for line in open("/kaggle/input/phones-data/cleaned_parallel_phones_val_phones.txt", "r", encoding="utf-8")]
val_tgt_lines = [line.strip() for line in open("/kaggle/input/phones-data/cleaned_parallel_phones_val_hindi.txt", "r", encoding="utf-8")]
train_data = [{"src": s, "tgt": t} for s, t in zip(train_src_lines, train_tgt_lines) if s and t]
num_val_samples = 300  # Adjust this number as needed
val_src_lines = val_src_lines[:num_val_samples]
val_tgt_lines = val_tgt_lines[:num_val_samples]
# =============================
val_data = [{"src": s, "tgt": t} for s, t in zip(val_src_lines, val_tgt_lines) if s and t]
raw_datasets = DatasetDict({
    'train': Dataset.from_list(train_data),
    'validation': Dataset.from_list(val_data)
})

# --- Tokenization Function ---
max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    # Set tokenizer language context correctly
    tokenizer.src_lang = source_lang_code # e.g., "en_XX"

    model_inputs = tokenizer(
        examples["src"],
        max_length=max_input_length,
        truncation=True,
        padding=False # Padding handled by DataCollator
    )

    # Setup the tokenizer for target language
    # Correct way for MBart: Use 'as_target_tokenizer' context manager
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["tgt"],
            max_length=max_target_length,
            truncation=True,
            padding=False # Padding handled by DataCollator
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# --- Apply Tokenization ---
print("Tokenizing datasets with extended tokenizer...")
tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)
print(tokenized_datasets)
# Check an example to see if phonetic tokens get correct IDs
print("Example tokenized input:", tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][0]['input_ids']))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64",

Map:   0%|          | 0/7337 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7337
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
})
Example tokenized input: ['en_XX', 'ʋ', '▁', 'ɛ', '▁', 'd̪', '▁', 'ə', '▁', '<WB>', '▁', 'd̪', '▁', 'ə', '▁', '<WB>', '▁', 'k', '▁', 'ə', '▁', 'm', '▁', 'p', '▁', 'ə', '▁', 'ɲ', '▁', 'i', '▁', 'z', '▁', '<WB>', '▁', 'f', '▁', 'ə', '▁', 'n', '▁', '<WB>', '▁', 'p', '▁', 'ə', '▁', 'dʒ', '▁', 'ɪ', '▁', 'ʃ', '▁', 'ə', '▁', 'n', '▁', '<WB>', '▁', 'ɪ', '▁', 'z', '▁', '<WB>', '▁', 'oː', '▁', 'k', '▁', 'eː', '▁', '<WB>', '▁', 'ɒː', '▁', '<WB>', '▁', 'n', '▁', 'ɒ', '▁', 'ʈ', '</s>']


In [8]:

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import torch
import numpy as np
!pip install evaluate
import evaluate
# --- compute_metrics function (use the corrected version from previous response) ---
metric = evaluate.load("bleu")
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
"""
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple): preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) # Use the extended tokenizer
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True) # Use the extended tokenizer
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    if "score" in result:
        result["bleu"] = round(result["score"], 4)
        result.pop("score")
    else: result["bleu"] = 0.0
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = round(np.mean(prediction_lens), 4)
    # print(f"\nRAW compute_metrics result: {result}\n") # Optional debug
    return result
# --- End compute_metrics ---
"""
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Basic preprocessing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    # Compute BLEU
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    bleu_score = result["bleu"] if "bleu" in result else 0.0

    # Output only BLEU and generation length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    return {
        "bleu": round(bleu_score, 4),
        "gen_len": round(np.mean(prediction_lens), 4)
    }


# --- Training Arguments (Adjust for Fine-tuning) ---
output_dir = "phonetic-hi-mbart-finetuned"
per_device_batch_size = 8 # May need to adjust based on GPU memory (MBart-large is big)
num_train_epochs = 20       # Start with fewer epochs for fine-tuning (e.g., 3-10)
learning_rate = 3e-5     # Typically smaller LR for fine-tuning (e.g., 2e-5 to 5e-5)
use_fp16 = torch.cuda.is_available()

args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_batch_size,
    per_device_eval_batch_size=per_device_batch_size,
    weight_decay=0.01,
    save_strategy="epoch",
    save_only_model=True,
    save_total_limit=1,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_strategy="epoch", # Log metrics after each epoch's eval
    fp16=use_fp16,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
    # Necessary for MBart when fine-tuning on specific language pairs
    generation_max_length=max_target_length, # Ensure generation matches max length used
    # report_to="none",
)

# --- Data Collator ---
# Use the extended tokenizer here as well
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, # Pass the extended tokenizer
    model=model,
    padding=True
)

# --- Trainer Initialization ---
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer, # Pass the extended tokenizer
    compute_metrics=compute_metrics
)



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/tmp/ipykernel_31/3312205103.py:94: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [9]:
import wandb
wandb.login(key="5c556254e04a65a22bedb25b2faac3fc1b5b5272")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saketvempaty to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [10]:
# --- Start Training ---
print("Starting fine-tuning...")
try:
    train_result = trainer.train()

    # --- Save Final Model & Metrics ---
    print("Fine-tuning finished.")
    trainer.save_model() # Saves the best model based on eval_bleu
    metrics = train_result.metricsc
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()

    # CRITICAL: Save the extended tokenizer with the fine-tuned model
    tokenizer.save_pretrained(output_dir)
    print(f"Fine-tuned model and EXTENDED tokenizer saved to {output_dir}")

except Exception as e:
    print(f"An error occurred during training: {e}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Starting fine-tuning...


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,3.163500,2.549330,0.065900,36.746700
2,2.210800,2.295098,0.118300,37.716700
3,1.728100,2.248982,0.125800,28.210000
4,1.350700,2.302347,0.144300,30.730000
5,1.037600,2.406799,0.132300,26.853300
6,0.779300,2.529534,0.143000,30.970000
7,0.576400,2.606610,0.147100,31.313300
8,0.413700,2.716183,0.139000,28.056700
9,0.291100,2.788279,0.139000,29.736700
10,0.206900,2.863130,0.146400,28.930000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Fine-tuning finished.
***** train metrics *****
  epoch                    =       20.0
  total_flos               = 37017372GF
  train_loss               =     0.6169
  train_runtime            = 3:28:01.26
  train_samples_per_second =     11.757
  train_steps_per_second   =      1.471
Fine-tuned model and EXTENDED tokenizer saved to phonetic-hi-mbart-finetuned
